In [2]:

import logging
import os
from dataclasses import dataclass

import numpy as np
import pandas as pd
import patoolib

from common.http_utils import download, url_file_name
from common.settings import DATASETS_PATH

DATASET_URL = 'C:\Users\verma\Downloads\us-east-2-preprocessed-task.csv'

DATASET_PATH = os.path.join(DATASETS_PATH, 'test')
DATASET_FILE_PATH = os.path.join(DATASET_PATH, url_file_name(DATASET_URL))





@dataclass()
class TestDataSet:
    ids: np.ndarray
    groups: np.ndarray
    horizons: np.ndarray
    values: np.ndarray

    @staticmethod
    def load(training: bool = True) -> 'TestDataSet':

        ids = []
        groups = []
        horizons = []
        values = []


        train = pd.read_csv(os.path.join(DATASET_PATH, f'{group.lower()}_in.csv'),
                                header=0, delimiter=",")
        test = pd.read_csv(os.path.join(DATASET_PATH, f'{group.lower()}_oos.csv'),
                               header=0, delimiter=",")

        horizons.extend(list(test.iloc[0].astype(int)))
        groups.extend([group] * len(train.columns))

        ids.extend(list(train.index))

        if training:
            dataset = train
        else:
            dataset = test

        values.extend([ts[:ts_length] for ts, ts_length in zip(dataset.values)])

        return TestDataSet(ids=np.array(ids),
                              groups=np.array(groups),
                              horizons=np.array(horizons),
                              values=np.array(values))

    @staticmethod
    def download():

        if os.path.isdir(DATASET_PATH):
            logging.info(f'skip: {DATASET_PATH} directory already exists.')
            return
        download(DATASET_URL, DATASET_FILE_PATH)
        patoolib.extract_archive(DATASET_FILE_PATH, outdir=DATASET_PATH)

    def to_hp_search_training_subset(self):
        return TestDataSet(ids=self.ids,
                              groups=self.groups,
                              horizons=self.horizons,
                              values=np.array([v[:-self.horizons[i]] for i, v in enumerate(self.values)]))


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (4168891403.py, line 12)